In [50]:
import os
import json
import pandas as pd
from pandas2arff import pandas2arff
from sklearn.preprocessing import LabelEncoder

def printDF(title, df):
    print "##############################\n    "+title+"    \n##############################\n"
    print "## Shape: ##"
    print df.shape
    print "\n## Missing Values per Column: ##"
    print df.isnull().sum()
    # print "\n## Show data: ##"
    # print df[0:12]
    print "############################## \n\n"

def createDF(file_name):
    # load data from json file
    #working_dir = os.path.dirname(os.path.realpath(__file__)) + '/'
    with open(file_name, "r") as f:
        found_jobs = f.read()
    data = json.loads(found_jobs)

    # normalize because of nested client data
    df = pd.io.json.json_normalize(data)
    df.columns = [c.replace('.', '_') for c in df.columns] # so we can access a column with "data_frame.client_reviews_count"
    return df

In [51]:
data_frame = createDF("found_jobs_4K.json")

printDF("Before changing data", data_frame)

##############################
    Before changing data    
##############################

## Shape: ##
(4779, 19)

## Missing Values per Column: ##
budget                                 588
category2                                0
client_country                           0
client_feedback                          0
client_jobs_posted                       0
client_past_hires                        0
client_payment_verification_status    3106
client_reviews_count                     0
date_created                             0
duration                              2730
id                                       0
job_status                               0
job_type                                 0
skills                                   0
snippet                                  0
subcategory2                             0
title                                    0
url                                      0
workload                               997
dtype: int64
####################

## Remove attributes which have as many values as there are examples

![Useless attributes](images/useless_attributes.jpg)

Removing the `id` attribute could however be problematic, since there it could be useful later, when trying to find similar jobs

In [52]:
unnecessary_columns = ["id", "category2", "job_status", "url"]
data_frame.drop(labels=unnecessary_columns, axis=1, inplace=True)

## Remove attributes which don't add much value but which have too many missing values

In [53]:
bad_columns = ["client_payment_verification_status"]
data_frame.drop(labels=bad_columns, axis=1, inplace=True)

## Remove examples which contribute missing values to attributes with < 30 missing attributes

In [54]:
max_few_missing = 30
columns_few_missing = list(data_frame.columns[data_frame.isnull().sum() < max_few_missing])
data_frame.dropna(subset=columns_few_missing, how='any')

,budget,client_country,client_feedback,client_jobs_posted,client_past_hires,client_reviews_count,date_created,duration,job_type,skills,snippet,subcategory2,title,workload
0,50.0,United Kingdom,5.000000,7,6,3,2017-05-09T18:27:32+0000,None,Fixed,"[data-scraping, web-scraping]",I am looking for some one experinced with webs...,Data Mining & Management,Extract Database from specific website,None
1,0.0,United States,5.000000,6,5,1,2017-05-07T02:35:05+0000,Less than 1 week,Hourly,[],Seeking a mathematician or statistician to hel...,Quantitative Analysis,Mathematical formula/ algorithm modelization,Less than 10 hrs/week
2,500.0,United States,5.000000,3,3,1,2017-05-06T21:36:46+0000,None,Fixed,"[google-analytics, google-tag-manager, javascr...",I currently have an issue with Google Analytic...,Other - Data Science & Analytics,Google Analytics Session Tracking Upgrade,None
3,0.0,United States,0.000000,4,4,2,2017-05-05T19:13:15+0000,Less than 1 week,Hourly,"[data-entry, internet-research, microsoft-exce...",Looking for someone who can quickly research c...,Data Mining & Management,NEED QUICKLY!! FAST RESPONSE WILL BE PRIORITIZ...,Less than 10 hrs/week
4,150.0,United States,5.000000,50,44,4,2017-05-04T17:37:11+0000,None,Fixed,"[data-visualization, excel-vba, microsoft-excel]",Add Pivot tables into dashboard of an existing...,Data Visualization,Excel Pivot tables,None
5,20.0,United States,4.984906,7,21,15,2017-05-04T15:49:47+0000,None,Fixed,"[data-entry, data-mining]",Looking for someone to locate email addresses ...,Data Mining & Management,Find Blogger E-mails,None
6,20.0,United States,5.000000,2,4,1,2017-05-04T03:56:25+0000,None,Fixed,"[excel-vba, microsoft-excel]","I have an array of roughly 42,000 x 2600 pairs...",Data Mining & Management,Calculate array in Excel - distance between tw...,None
7,400.0,United States,5.000000,4,4,3,2017-05-03T14:30:07+0000,None,Fixed,[],I need an application or script written that c...,Data Extraction / ETL,Extract Data From Zillow,None
8,35.0,United States,5.000000,82,72,55,2017-05-03T13:42:03+0000,None,Fixed,"[excel-vba, microsoft-excel]",Look for someone who can help me quickly build...,Data Mining & Management,Microsoft Excel Equation Expert,None
9,100.0,United States,4.983217,7,21,11,2017-05-03T13:41:22+0000,None,Fixed,"[data-entry, data-mining]",Looking for someone to go through excel spread...,Data Mining & Management,Blogger Data and E-mail Mining,None


## Set feedback to None on examples where the client has not yet received a review

In [55]:
data_frame.loc[data_frame.client_reviews_count == 0, 'client_feedback'] = None

## Set budget to None if it is 0 (jobs without a budget cannot exist)

In [56]:
data_frame.loc[data_frame.budget == 0, 'budget'] = None

## Convert nominal attributes to numerical

This is done to make them available to all regression models - especially models that would not accept nominal attributes.

The following attributes seem interesting for this conversion:
![Nominal attributes to be converted](images/nominal_to_numeric.jpg)

In [57]:
clientCountryEncoder = LabelEncoder()
data_frame['client_country'] = clientCountryEncoder.fit_transform(data_frame['client_country'].astype('str'))
durationEncoder = LabelEncoder()
data_frame['duration'] = durationEncoder.fit_transform(data_frame['duration'].astype('str'))
jobTypeEncoder = LabelEncoder()
data_frame['job_type'] = jobTypeEncoder.fit_transform(data_frame['job_type'].astype('str'))
subcategory2Encoder = LabelEncoder()
data_frame['subcategory2'] = subcategory2Encoder.fit_transform(data_frame['subcategory2'].astype('str'))
workloadEncoder = LabelEncoder()
data_frame['workload'] = workloadEncoder.fit_transform(data_frame['workload'].astype('str'))

## Convert the `date_created` attribute into a delta between the date created and the time now and rename it to `posting_age`

In [58]:
import iso8601
import datetime
from dateutil import tz

now = datetime.datetime.now(tz.gettz("Europe/Zurich"))
dates = []
for date in data_frame['date_created']:
    dates.append(now - iso8601.parse_date(date))

data_frame['date_created'] = [date.seconds for date in dates]
data_frame.rename(columns={'date_created': 'posting_age'}, inplace=True)

In [59]:
printDF("After changing data", data_frame)

##############################
    After changing data    
##############################

## Shape: ##
(4779, 14)

## Missing Values per Column: ##
budget                  2049
client_country             0
client_feedback          133
client_jobs_posted         0
client_past_hires          0
client_reviews_count       0
posting_age                0
duration                   0
job_type                   0
skills                     0
snippet                    0
subcategory2               0
title                      0
workload                   0
dtype: int64
############################## 


